### Tools

Key concepts
+ Tools are a way to encapsulate a function and its schema in a way that can be passed to a chat model.

Create tools using the @tool decorator, which simplifies the process of tool creation, supporting the following:

Automatically infer the tool's name, description and expected arguments, while also supporting customization.

Defining tools that return artifacts (e.g. images, dataframes, etc.)

Hiding input arguments from the schema (and hence from the model) using injected tool arguments.



<!-- The key attributes that correspond to the tool's schema:
1-name: The name of the tool.
2-description: A description of what the tool does.
3-args: Property that returns the JSON schema for the tool's arguments. -->



### Create tools using the @tool decorator

In [1]:
from langchain_core.tools import tool

@tool
def add(a:int,b:int)-> int:
    """"Add two numbers"""
    return a + b

In [2]:
add.invoke({
    "a":2,
    "b":3
})

5

In [3]:
print(add.name)
print(add.description)
print(add.args)

add
"Add two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


### 2 -DuckDuckGo Search Prebuild tools in langchain
This guide shows over how to use the DuckDuckGo search component.

In [4]:
from langchain_community.tools import DuckDuckGoSearchRun,DuckDuckGoSearchResults

search = DuckDuckGoSearchResults(output_format="list")
search.invoke("Obama")



/home/jamxt/Desktop/advancepython/GenAI_With_Langchain/venv/lib/python3.12/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


[{'snippet': '贝拉克·侯赛因·奥巴马（Barack Hussein Obama），1961年8月4日出生于美国夏威夷州檀香山，美国民主党籍政治家，第44任美国总统，美国历史上第一位非洲裔总统。',
  'title': '贝拉克·侯赛因·奥巴马 - 知乎',
  'link': 'https://www.zhihu.com/topic/19585074/intro'},
 {'snippet': '美国第44任总统奥巴马的全称是 巴拉克·侯赛因·奥巴马 二世（Barack Hussein Obama II）。称其为二世，是为了与和他姓名完全相同的父亲（巴拉克·侯赛因·奥巴马一世）相区别。在西方许 …',
  'title': '奥巴马的中间名为何叫侯赛因？ - 知乎',
  'link': 'https://www.zhihu.com/question/20733642'},
 {'snippet': '出镜率太高，而且打破了卸任总统的潜规则。 按以往的惯例，美国卸任总统应少在公众面前出镜，如非必要，不应太多干预当前政治。 你可以捞钱、可以走穴、可以写回忆录，不嫌丢人的 …',
  'title': '怎样评价奥巴马？ - 知乎',
  'link': 'https://www.zhihu.com/question/54942704'},
 {'snippet': 'Sep 30, 2021 · 这是奥巴马2004年在民主党全国代表大会上竞选联邦参议员的主题演讲——《无畏的希望》（Audacity of Hope）。 这篇慷慨激昂的演讲让奥巴马一举成名，成为全美知名的政 …',
  'title': '奥巴马的成名演讲：《无畏的希望》。 非常值得一听 ...',
  'link': 'https://www.zhihu.com/zvideo/1426695736242769920'}]

In [5]:
# !pip install -U duckduckgo-search

In [6]:
from langchain_core.tools import tool

# add tool
@tool
def add(a: int, b: int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b

In [7]:
import os
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI


# loading env varables
from dotenv import load_dotenv
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

# create the chatllm 

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
)
tools = [add, multiply]




In [8]:
llm_with_tools= llm.bind_tools(tools)

In [9]:
from langchain_core.messages import HumanMessage


In [22]:

query = "What is 3 * 12? Also, what is 11 + 49?"

messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

print(ai_msg.tool_calls)

[{'name': 'multiply', 'args': {'a': 3.0, 'b': 12.0}, 'id': 'dea30842-fd64-4c63-9a87-368eeae77ba5', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11.0, 'b': 49.0}, 'id': '51d70eb9-efc4-479a-855a-8fd27aaa9ab8', 'type': 'tool_call'}]


In [23]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'add', 'arguments': '{"a": 11.0, "b": 49.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--fcbfc647-56c0-409b-820d-12c6186773b8-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 12.0}, 'id': 'dea30842-fd64-4c63-9a87-368eeae77ba5', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11.0, 'b': 49.0}, 'id': '51d70eb9-efc4-479a-855a-8fd27aaa9ab8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 77, 'output_tokens': 10, 'total_tokens': 87, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='36', name='multiply', tool_call_id='dea30842-fd64-4c63-9a87-368eeae77ba5'),
 ToolMessage(content='60', name='add', tool_call_id='51d70eb9-efc4-479a-855a-8fd27aaa9ab8')]

In [24]:
messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'add', 'arguments': '{"a": 11.0, "b": 49.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--fcbfc647-56c0-409b-820d-12c6186773b8-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 12.0}, 'id': 'dea30842-fd64-4c63-9a87-368eeae77ba5', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11.0, 'b': 49.0}, 'id': '51d70eb9-efc4-479a-855a-8fd27aaa9ab8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 77, 'output_tokens': 10, 'total_tokens': 87, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='36', name='multiply', tool_call_id='dea30842-fd64-4c63-9a87-368eeae77ba5'),
 ToolMessage(content='60', name='add', tool_call_id='51d70eb9-efc4-479a-855a-8fd27aaa9ab8')]

In [25]:
res = llm_with_tools.invoke(messages)

In [26]:
res.content

'3 * 12 is 36. 11 + 49 is 60.'

In [28]:
llm_with_tools.invoke("what is ai")

AIMessage(content='Artificial intelligence (AI) is a broad field encompassing the development of computer systems that can perform tasks that typically require human intelligence. These tasks include learning, problem-solving, decision-making, and perception.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--60a27e38-6e26-445f-99c4-79b3e0e45668-0', usage_metadata={'input_tokens': 59, 'output_tokens': 41, 'total_tokens': 100, 'input_token_details': {'cache_read': 0}})